In [29]:
from pathlib import Path

import pandas as pd

pd.options.display.max_columns = None

# The importance of sandboxes

I've worked with SQL regularly over the past decade, but over the first half of that decade (and despite extensive formal knowledge of normalization, data modeling, and other database-related topics) I essentially only used SQL to extract data and use Python or R to do analysis. I only had access to production databases and I didn't know enough about how database management system work to risk experiments, and as a result, I learned at a glacial pace. But in 2018, I stumbled into a project interesting enough to motivate me to install PostGIS on a personal machine and freed from the fear of accidentally taking down a production system (and the power of a superuser), I was able to lift up the hood and see how the machine worked, and this enabled me to learn and build with PostgreSQL + PostGIS (a geospatial PostgreSQL extension) far faster.

In this post, I show how to use Docker to set up a PostGIS database and experiment with it.

## What is Docker

Docker and Docker Compose[1^] enable you to import and run complicated applications in just a few lines of code.

The internals of docker are really interesting, but for this post, all you need to know these things:

1. A docker **image** is like a blueprint of an application, and it's defined in **Dockerfiles**.
2. A docker **container** is a runnable instance of the application, built from the instructions in the blueprint.
3. You can configure your application in a `docker-compose.yml` file.

### Our Dockerfile
The first one may sound complicated, as the contents of our Dockerfile show, this can only take one line of code (from us). That's possible because Dockerfiles can build an image based on another image, and the developers of many open source projects (e.g. Ubuntu, PostgreSQL/PostGIS, MySQL, Go, nginx, etc) public official images on [Docker Hub](https://registry.hub.docker.com/search?q=). Our Dockerfile pulls the [postgis/postgis image](https://registry.hub.docker.com/r/postgis/postgis) (translated, it pulls the `postgis` base image from the `postgis` organization), and then adds nothing else. So in one line, we indicate that we want the `postgis/postgis` image that has the tag "15-3.3" (which they've intuitively used to indicate the image provides a PostGIS database that has version 15-3.3).

In [2]:
#| code-fold: true
#| code-summary: The full contents of our Dockerfile

!cat db_context/Dockerfile

FROM postgis/postgis:15-3.3

### Our docker-compose.yml file

And this is our `docker-compose.yml` file. The `docker-compose.yml` file defines the services, and any networks, volumes, configs, and/or secrets/environment variables your system needs to work. The file below defines one service (named `postgis`) and one volume (named `sandbox_postgis_data`).

In [3]:
#| code-fold: true
#| code-summary: The contents of our docker-compose.yml

!cat docker-compose.yml

version: '3.9'

services:
  postgis:
    image: sandbox_postgis:15.3.3
    build:
      context: ./db_context
      dockerfile: Dockerfile
    ports:
      - 54321:5432
    environment:
      POSTGRES_DB: db_name
      POSTGRES_USER: db_username
      POSTGRES_PASSWORD: db_password
    volumes:
      - sandbox_postgis_data:/var/lib/postgresql/data

volumes:
  sandbox_postgis_data:


There are 3 top level elements in this file: `version`, `services`, and `volumes`.

* `version` indicates the version of the Docker Compose specification; I don't think I've ever set it to anything other than 3.9.
* `services` defines configurations for each container your system needs.
* `volumes` defines persistant data stores that can be shared by different services.

#### Services
The `postgis` service has five elements: `image`, `build`, `ports`, `environment`, and `volumes`.

The `image` and `build` elements define the docker image to use; `image` defines both the name ("sandbox_postgis") and tag ("15.5.3") for the docker image, and `build` defines the Dockerfile to build into an image as well as the context to build into the docker image.

The `port` element defines a connection from a port on the host machine (host post 54321) to a port into the container (container port 5432, the default for PostgreSQL). We'll use that later to connect to the database.

The `volumes` element (in the `postgis` service) defines a persistant storage volume that will hold the data in our database. Without this, our database would reset every time we restart the system [^2].

And the `environment` element enables you to set environment variables in the container. Here, we pass in environment variables `POSTGRES_DB`, `POSTGRES_USER`, and `POSTGRES_PASSWORD` which are used to name the database (in this case, it's named `db_name`) and create a superuser (in this case having username `db_username` and password `db_password`) when the database is first created in a new volume.

### Building the image(s) used in our system

The first time you build the image defined by your Dockerfile, docker will read your Dockerfile(s), download all layers of the base image(s) (defined in lines starting with `FROM `), process each subsequent instruction into a layer, cache layers, and then bind the layers into an image. This is a template for creating containers.

The second time you run this command (assuming no changes have been made to the `docker-compose.yml` file, Dockerfile(s), or any other files the Dockerfile references), all layers will just be pulled from cache, producing much smaller output (like what's shown below).

In [4]:
#| code-fold: true
#| code-summary: The output produced while building our images

!docker compose build

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/4)                                                         
 => [postgis internal] load build definition from Dockerfile               0.0s
 => => transferring dockerfile: 64B                                        0.0s
 => [postgis internal] load .dockerignore                                  0.0s
 => => transferring context: 2B                                            0.0s
 => [postgis internal] load metadata for docker.io/postgis/postgis:15-3.3  0.1s
 => [postgis auth] postgis/postgis:pull token for registry-1.docker.io     0.0s
[+] Building 0.3s (3/4)                                                         
 => [postgis internal] load build definition from Dockerfile               0.0s
 => => transferring dockerfile: 64B                                        0.0s
 => [postgis internal] load .dockerignore                                  0.0s
 => => transferring context: 2B      

In [38]:
#| code-fold: true
#| code-summary: Shows that our system's one container is not running yet

!docker ps -f name=sandbox-postgis

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


`docker ps` shows running containers, and the `-f name=...` option allows us to filter to running containers with a name containing the entered string. Currently, "sandbox-postgis" isn't part of the name of any running container, so let's spin one up.

In [39]:
#| code-fold: false
#| code-summary: The command to start up our system

!docker compose up -d

[+] Running 1/0
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠋ Container 015_docker_postgres_sandbox-postgis-1  Creating               0.0s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠙ Container 015_docker_postgres_sandbox-postgis-1  Creating               0.1s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠹ Container 015_docker_postgres_sandbox-postgis-1  Creating               0.2s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠸ Container 015_docker_postgres_sandbox-postgis-1  Creating               0.3s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠼ Container 015_docker_postgres_sandbox-postgis-1  Creating               0.4s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠴ Container 

In [40]:
#| code-fold: true
#| code-summary: Shows that our system's one container is running now

!docker ps -f name=sandbox-postgis

CONTAINER ID   IMAGE                    COMMAND                  CREATED         STATUS                  PORTS                                         NAMES
8513cc1ff07d   sandbox_postgis:15.3.3   "docker-entrypoint.s…"   2 seconds ago   Up Less than a second   0.0.0.0:54321->5432/tcp, :::54321->5432/tcp   015_docker_postgres_sandbox-postgis-1


## Sending commands to our database

Now we have our database up and running on our local system, but now we need to connect to it. There are countless ways to connect to databases, but I want to focus on the SQL, so I've written just enough to allow me to connect to our datasbase (via my `get_db_connection()` function) and execute a SQL query/statement (via my `execute_query()` function.

Note that `get_db_connection()` uses the `POSTGRES_DB`, `POSTGRES_USER`, and `POSTGRES_PASSWORD` environment variables as well as port number 54321, all of which we set in our `docker-compose.yml` file.

In [41]:
#| code-fold: false
#| code-summary: Our extremely thi

from contextlib import contextmanager
from typing import Union

import pandas as pd
import psycopg2


@contextmanager
def get_db_connection():
    conn = psycopg2.connect(
        dbname="db_name",
        user="db_username",
        password="db_password",
        host="localhost",
        port=54321
    )
    try:
        yield conn
    finally:
        conn.close()

def execute_query(query: str) -> Union[pd.DataFrame, None]:
    with get_db_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            rows = cur.fetchall()
            if rows:
                col_names = [desc[0] for desc in cur.description]
                return pd.DataFrame(rows, columns=col_names)

In [20]:
schemas = execute_query(query="""
    SELECT nspname AS schema_name
    FROM pg_namespace;
""")
schemas

,schema_name
0,pg_toast
1,pg_catalog
2,public
3,information_schema
4,topology
5,tiger
6,tiger_data


In [23]:
execute_query(query="""SELECT current_user;""")

,current_user
0,db_username


In [24]:
pg_databases_df = execute_query(
    query="""
        SELECT *
        FROM pg_database;
    """
)
pg_databases_df

,oid,datname,datdba,encoding,datlocprovider,datistemplate,datallowconn,datconnlimit,datfrozenxid,datminmxid,dattablespace,datcollate,datctype,daticulocale,datcollversion,datacl
0,5,postgres,10,6,c,False,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,None
1,16384,db_name,10,6,c,False,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,None
2,1,template1,10,6,c,True,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,"{=c/db_username,db_username=CTc/db_username}"
3,4,template0,10,6,c,True,False,-1,717,1,1663,en_US.utf8,en_US.utf8,None,None,"{=c/db_username,db_username=CTc/db_username}"
4,16385,template_postgis,10,6,c,True,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,None


In [25]:
pg_namespaces_df = execute_query(
    query="""
        SELECT *
        FROM pg_namespace;
    """
)
pg_namespaces_df

,oid,nspname,nspowner,nspacl
0,99,pg_toast,10,None
1,11,pg_catalog,10,"{db_username=UC/db_username,=U/db_username}"
2,2200,public,6171,"{pg_database_owner=UC/pg_database_owner,=U/pg_..."
3,13186,information_schema,10,"{db_username=UC/db_username,=U/db_username}"
4,19052,topology,10,"{db_username=UC/db_username,=U/db_username}"
5,19226,tiger,10,None
6,19482,tiger_data,10,None


In [26]:
pg_roles_df = execute_query(
    query="""
        SELECT *
        FROM pg_roles;
    """
)
pg_roles_df

,rolname,rolsuper,rolinherit,rolcreaterole,rolcreatedb,rolcanlogin,rolreplication,rolconnlimit,rolpassword,rolvaliduntil,rolbypassrls,rolconfig,oid
0,pg_database_owner,False,True,False,False,False,False,-1,********,None,False,None,6171
1,pg_read_all_data,False,True,False,False,False,False,-1,********,None,False,None,6181
2,pg_write_all_data,False,True,False,False,False,False,-1,********,None,False,None,6182
3,pg_monitor,False,True,False,False,False,False,-1,********,None,False,None,3373
4,pg_read_all_settings,False,True,False,False,False,False,-1,********,None,False,None,3374
5,pg_read_all_stats,False,True,False,False,False,False,-1,********,None,False,None,3375
6,pg_stat_scan_tables,False,True,False,False,False,False,-1,********,None,False,None,3377
7,pg_read_server_files,False,True,False,False,False,False,-1,********,None,False,None,4569
8,pg_write_server_files,False,True,False,False,False,False,-1,********,None,False,None,4570
9,pg_execute_server_program,False,True,False,False,False,False,-1,********,None,False,None,4571


In [30]:
pg_classes_df = execute_query(
    query="""
        SELECT *
        FROM pg_class;
    """
)
pg_classes_df

,oid,relname,relnamespace,reltype,reloftype,relowner,relam,relfilenode,reltablespace,relpages,reltuples,relallvisible,reltoastrelid,relhasindex,relisshared,relpersistence,relkind,relnatts,relchecks,relhasrules,relhastriggers,relhassubclass,relrowsecurity,relforcerowsecurity,relispopulated,relreplident,relispartition,relrewrite,relfrozenxid,relminmxid,relacl,reloptions,relpartbound
0,18312,geometry_dump,2200,18314,0,10,0,0,0,0,-1.0,0,0,False,False,p,c,2,0,False,False,False,False,False,True,n,False,0,0,0,None,None,None
1,19439,idx_tiger_featnames_lname,19226,0,0,10,403,19439,0,1,0.0,0,0,False,False,p,i,1,0,False,False,False,False,False,True,n,False,0,0,0,None,None,None
2,18323,pg_toast_18319,99,0,0,10,2,18323,0,0,-1.0,0,0,True,False,p,t,3,0,False,False,False,False,False,True,n,False,0,732,1,None,None,None
3,18324,pg_toast_18319_index,99,0,0,10,403,18324,0,1,0.0,0,0,False,False,p,i,2,0,False,False,False,False,False,True,n,False,0,0,0,None,None,None
4,19440,idx_tiger_featnames_tlid_statefp,19226,0,0,10,403,19440,0,1,0.0,0,0,False,False,p,i,2,0,False,False,False,False,False,True,n,False,0,0,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,19584,pagc_rules_id_seq,19226,0,0,10,0,19584,0,1,1.0,0,0,False,False,p,S,3,0,False,False,False,False,False,True,n,False,0,0,0,None,None,None
589,19590,pg_toast_19585,99,0,0,10,2,19590,0,0,-1.0,0,0,True,False,p,t,3,0,False,False,False,False,False,True,n,False,0,735,1,None,None,None
590,19591,pg_toast_19585_index,99,0,0,10,403,19591,0,1,0.0,0,0,False,False,p,i,2,0,False,False,False,False,False,True,n,False,0,0,0,None,None,None
591,19592,pagc_rules_pkey,19226,0,0,10,403,19592,0,27,4354.0,0,0,False,False,p,i,1,0,False,False,False,False,False,True,n,False,0,0,0,None,None,None


In [32]:
pg_classes_df.loc[pg_classes_df["relname"].str.contains("geometry")]

,oid,relname,relnamespace,reltype,reloftype,relowner,relam,relfilenode,reltablespace,relpages,reltuples,relallvisible,reltoastrelid,relhasindex,relisshared,relpersistence,relkind,relnatts,relchecks,relhasrules,relhastriggers,relhassubclass,relrowsecurity,relforcerowsecurity,relispopulated,relreplident,relispartition,relrewrite,relfrozenxid,relminmxid,relacl,reloptions,relpartbound
0,18312,geometry_dump,2200,18314,0,10,0,0,0,0,-1.0,0,0,False,False,p,c,2,0,False,False,False,False,False,True,n,False,0,0,0,None,None,None
9,18857,geometry_columns,2200,18859,0,10,0,0,0,0,-1.0,0,0,False,False,p,v,7,0,True,False,False,False,False,True,n,False,0,0,0,"{db_username=arwdDxt/db_username,=r/db_username}",None,None
23,19083,topogeometry,19052,19085,0,10,0,0,0,0,-1.0,0,0,False,False,p,c,4,0,False,False,False,False,False,True,n,False,0,0,0,None,None,None


In [34]:
pg_data_types_df = execute_query(
    query="""
        SELECT *
        FROM pg_type;
    """
)
pg_data_types_df

,oid,typname,typnamespace,typowner,typlen,typbyval,typtype,typcategory,typispreferred,typisdefined,typdelim,typrelid,typsubscript,typelem,typarray,typinput,typoutput,typreceive,typsend,typmodin,typmodout,typanalyze,typalign,typstorage,typnotnull,typbasetype,typtypmod,typndims,typcollation,typdefaultbin,typdefault,typacl
0,16,bool,11,10,1,True,b,B,True,True,",",0,-,0,1000,boolin,boolout,boolrecv,boolsend,-,-,-,c,p,False,0,-1,0,0,None,None,None
1,17,bytea,11,10,-1,False,b,U,False,True,",",0,-,0,1001,byteain,byteaout,bytearecv,byteasend,-,-,-,i,x,False,0,-1,0,0,None,None,None
2,18,char,11,10,1,True,b,Z,False,True,",",0,-,0,1002,charin,charout,charrecv,charsend,-,-,-,c,p,False,0,-1,0,0,None,None,None
3,19,name,11,10,64,False,b,S,False,True,",",0,raw_array_subscript_handler,18,1003,namein,nameout,namerecv,namesend,-,-,-,c,p,False,0,-1,0,950,None,None,None
4,20,int8,11,10,8,True,b,N,False,True,",",0,-,0,1016,int8in,int8out,int8recv,int8send,-,-,-,d,p,False,0,-1,0,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,19566,_pagc_gaz,19226,10,-1,False,b,A,False,True,",",0,array_subscript_handler,19567,0,array_in,array_out,array_recv,array_send,-,-,array_typanalyze,d,x,False,0,-1,0,0,None,None,None
715,19577,pagc_lex,19226,10,-1,False,c,C,False,True,",",19575,-,0,19576,record_in,record_out,record_recv,record_send,-,-,-,d,x,False,0,-1,0,0,None,None,None
716,19576,_pagc_lex,19226,10,-1,False,b,A,False,True,",",0,array_subscript_handler,19577,0,array_in,array_out,array_recv,array_send,-,-,array_typanalyze,d,x,False,0,-1,0,0,None,None,None
717,19587,pagc_rules,19226,10,-1,False,c,C,False,True,",",19585,-,0,19586,record_in,record_out,record_recv,record_send,-,-,-,d,x,False,0,-1,0,0,None,None,None


In [36]:
pg_data_types_df.loc[pg_data_types_df["typname"].str.contains("geometry")]

,oid,typname,typnamespace,typowner,typlen,typbyval,typtype,typcategory,typispreferred,typisdefined,typdelim,typrelid,typsubscript,typelem,typarray,typinput,typoutput,typreceive,typsend,typmodin,typmodout,typanalyze,typalign,typstorage,typnotnull,typbasetype,typtypmod,typndims,typcollation,typdefaultbin,typdefault,typacl
613,18012,geometry,2200,10,-1,False,b,U,False,True,:,0,-,0,18020,geometry_in,geometry_out,geometry_recv,geometry_send,geometry_typmod_in,geometry_typmod_out,geometry_analyze,d,m,False,0,-1,0,0,None,None,None
614,18020,_geometry,2200,10,-1,False,b,A,False,True,:,0,array_subscript_handler,18012,0,array_in,array_out,array_recv,array_send,geometry_typmod_in,geometry_typmod_out,array_typanalyze,d,x,False,0,-1,0,0,None,None,None
623,18314,geometry_dump,2200,10,-1,False,c,C,False,True,",",18312,-,0,18313,record_in,record_out,record_recv,record_send,-,-,-,d,x,False,0,-1,0,0,None,None,None
624,18313,_geometry_dump,2200,10,-1,False,b,A,False,True,",",0,array_subscript_handler,18314,0,array_in,array_out,array_recv,array_send,-,-,array_typanalyze,d,x,False,0,-1,0,0,None,None,None
633,18859,geometry_columns,2200,10,-1,False,c,C,False,True,",",18857,-,0,18858,record_in,record_out,record_recv,record_send,-,-,-,d,x,False,0,-1,0,0,None,None,None
634,18858,_geometry_columns,2200,10,-1,False,b,A,False,True,",",0,array_subscript_handler,18859,0,array_in,array_out,array_recv,array_send,-,-,array_typanalyze,d,x,False,0,-1,0,0,None,None,None
639,19085,topogeometry,19052,10,-1,False,c,C,False,True,",",19083,-,0,19084,record_in,record_out,record_recv,record_send,-,-,-,d,x,False,0,-1,0,0,None,None,None
640,19084,_topogeometry,19052,10,-1,False,b,A,False,True,",",0,array_subscript_handler,19085,0,array_in,array_out,array_recv,array_send,-,-,array_typanalyze,d,x,False,0,-1,0,0,None,None,None


[^1] Before you can use docker and compose, you have to install [docker](https://docs.docker.com/engine/install/) and [compose](https://docs.docker.com/compose/install/). I installed the docker engine and compose, but it looks like the developers at Docker Inc. guide people towards installing their Docker Desktop client. In either setup, docker should still become available to you via the command line, so instructions in this post should work.

[^2] Docker containers are designed to be a replicable instance of an image, so when you shut down your application, your containers are removed and new ones are created next time you start it up. This is great for reproducibility (you always get a new, clean instance based on your image), but you don't want the data you ingest into your database to get wiped every time you shut down your system, so you can define a persistent **volume** that will live on in the host system.